In [6]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

corpus = [
    '아 더빙.. 진짜 짜증나네요 목소리',
    '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
    '너무재밓었다그래서보는것을추천한다',
    '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
]

okt = Okt()
tokenized_corpus = [' '.join(okt.morphs(text)) for text in corpus]
tokenized_corpus

okt.morphs(tokenized_corpus[0])



['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

1. okt().morphs 토큰화를 먼저하고 저장
2. vectorizer도 저장된 토큰에 fit_transform하고 저장

그러면 모델학습 테스트 데이터
1. okt().morphs 토큰화를 먼저하고
2. 불러온 vectorizer객체에 transform()해서 model.predict에 넣기


In [1]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리 해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')



In [2]:
import pandas as pd
#x,y값을 추출후 데이터프레임에 넣는다.
# X 값 추출 피쳐feature, y값 추출target 

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [3]:
# df_train.iloc[:,0]
from konlpy.tag import Okt

#문장을 토큰화한것만 따로 저장
# tokenized_data =[]

# for i in df_train.iloc[:,0]:
#     tokenized_data.append(Okt().morphs( i ))

In [4]:
import pickle

#토근화된 데이터를 저장하였음
# with open('tokenized_data.pkl','wb') as f:
#     pickle.dump(tokenized_data,f)

#토큰화된 데이터 불러오기
# with open('tokenized_data.pkl','rb') as f:
#     tokenized_data_loaded = pickle.load(f)

# tokenized_data_loaded

In [5]:
#라벨별로 분리해본것
bad = df_train[df_train.iloc[:,1]==0]
good = df_train[df_train.iloc[:,1]==1]


df_train = df_train[::4]

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer=TfidfVectorizer(tokenizer=Okt().morphs)
X = vectorizer.fit_transform( df_train.iloc[:,0])
y = df_train.iloc[:,1]


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [6]:
df_train[df_train.iloc[:,1]==0]  #18678 
df_train[df_train.iloc[:,1]==1]  #18822
df_train.shape

(37500, 2)

In [7]:
# 토큰화하고 빈도수(tfidf)에 기반한 벡터표현이 생성된 X를 따로저장
from scipy.sparse import save_npz ,load_npz

#vectorizer.fit_transform한것을 희소행렬로로 저장
# save_npz('37500개_vectorized_data.npz', X)

#저장된 희소행렬을 불러오기
# X_loaded = load_npz('37500개_vectorized_data.npz')

#이렇게 불러온 X_loaded는 바로 모델학습에 사용가능


In [8]:
import numpy as np
X_train_dense = np.expand_dims( X.toarray(), axis=-1 )
# X_train_dense.shape
# X_loaded.shape[1]
# X_loaded.toarray().shape[1]

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization , Input, Dropout
# from tensorflow.keras import Input
# 모델 구성deep nueral network
# input_layer = Input(shape=(X_loaded.shape[1],), sparse=True)

#flatten 사용하는것 알아보기

model = Sequential([
    # input_layer,
    Input(shape=(X_train_dense.shape[1],)),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(X.toarray(), y, epochs = 10, batch_size = 500, validation_split=0.2)  #0.8417
model.fit(X.toarray(), y, epochs = 10, batch_size = 20, validation_split=0.3)  #0.8417

c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/10
1313/1313 [==============================] - 44s 30ms/step - loss: 0.5788 - accuracy: 0.7039 - val_loss: 0.4331 - val_accuracy: 0.8047
Epoch 2/10
1313/1313 [==============================] - 30s 23ms/step - loss: 0.4476 - accuracy: 0.7979 - val_loss: 0.4013 - val_accuracy: 0.8164
Epoch 3/10
1313/1313 [==============================] - 28s 22ms/step - loss: 0.4056 - accuracy: 0.8261 - val_loss: 0.3935 - val_accuracy: 0.8208
Epoch 4/10
1313/1313 [==============================] - 29s 22ms/step - loss: 0.3737 - accuracy: 0.8421 - val_loss: 0.3940 - val_accuracy: 0.8219
Epoch 5/10
1313/1313 [==============================] - 26s 20ms/step - loss: 0.3584 - accuracy: 0.8526 - val_loss: 0.4147 - val_accuracy: 0.8083
Epoch 6/10
1313/1313 [==============================] - 29s 22ms/step - loss: 0.3678 - accuracy: 0.8474 - val_loss: 0.3993 - val_accuracy: 0.8182
Epoch 7/10
1313/1313 [==============================] - 29s 22ms/step - loss: 0.3398 - accuracy: 0.8645 - val_loss: 0.4036 -

In [10]:
test = ['정말 별로 입니다. 이게뭐니']

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
print(predictions)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round( predictions ).astype(int))

1/1 [==============================] - 0s 464ms/step
[[0.03403642]]
예측 결과: [[0]]


닉네임기준으로 
카톡대화내용을 기준으로 테스트 문장을 여러개 불러와서 예측 후 그 값의 비율을 계산 출력해준다.

In [11]:
# model.save('기본 DNN 적용모델.h5')